# Convertible Bond Strategy - Factor 0001

## 1. Import Library

In [5]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [6]:
sd = '2018-12-31'
ed = '2026-02-13'

In [7]:
sql_factor = f"""
SELECT
    date,
    instrument,
    close + conversion_premium_rate AS factor 
FROM cn_cbond_bar1d JOIN cn_cbond_analyze_metric USING (date, instrument)
WHERE bond_balance > 1.5 
AND remaining_days > 120
"""

In [8]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 10
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
    WHERE is_month_end_trade = 1
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [9]:
df_strategy = dai.query(sql_trade, filters={"date":[sd,ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2019-01-31,123012.SZ,100.815133,1.0,0.1
1,2019-01-31,123018.SZ,100.870331,2.0,0.1
2,2019-01-31,113524.SH,102.938496,3.0,0.1
3,2019-01-31,113016.SH,103.816671,4.0,0.1
4,2019-01-31,128049.SZ,103.979648,5.0,0.1
...,...,...,...,...,...
845,2026-01-30,127022.SZ,137.583952,6.0,0.1
846,2026-01-30,113676.SH,139.434600,7.0,0.1
847,2026-01-30,110094.SH,139.947019,8.0,0.1
848,2026-01-30,110092.SH,140.079106,9.0,0.1


## 3. Backtest Strategy

In [10]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""close""",
    order_price_field_sell="""close""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 21:01:54] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 21:01:54] [info     ] 2019-01-31, 2026-01-30, , , instruments=203
[2026-02-18 21:01:55] [info     ] bigtrader module V2.2.0
[2026-02-18 21:01:55] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-18 21:01:59] [info     ] backtest done, raw_perf_ds:dai.DataSource("_a38b91561fa24c858d1a035f794f418e")


[2026-02-18 21:02:00] [info     ] bigtrader.v34 运行完成 [5.882s].
